# <center> Линейная алгебра в контексте линейных методов. Практика.

## <center> Прогнозирование выработки газа на скважинах.

## Постановка задачи

У Василия, основателя компании «Газ-Таз-Ваз-Нефть», дела идут в гору: у него уже функционирует 200 скважин для добычи газа. В этом году он открывает 30 новых скважин. Однако в целях оптимизации расходов и повышения дохода Василию необходимо оценить, сколько денег будет приносить ему каждая из скважин, а также понять, какие факторы (параметры скважин) потенциально сильнее всего повлияют на объём добычи газа. Для этого Василий решил нанять вас как специалиста в области Data Science.

Василий представляет вам набор данных о добыче газа на своих скважинах. Файл с данными вы можете скачать на платформе.

**Признаки в данных:**

* Well — идентификатор скважины;
* Por — пористость скважины (%);
* Perm — проницаемость скважины;
* AI — акустический импеданс ($кг/м^2 * 10^6$);
* Brittle — коэффициент хрупкости скважины (%);
* TOC — общий органический углерод (%);
* VR — коэффициент отражения витринита (%);
* Prod — добыча газа в сутки (млн. кубических футов).

**Ваша задача** — построить регрессионную модель, которая прогнозирует выработку газа на скважине (целевой признак — Prod) на основе остальных характеристик скважины, и проинтерпретировать результаты вашей модели.

Разделим задачу на две части:
* В первой мы построим простейшую модель линейной регрессии, проанализируем результаты её работы и выберем наиболее значимые факторы для прогнозирования.
* Во второй мы займёмся построением модели полиномиальной регрессии с регуляризацией и посмотрим на итоговые результаты моделирования.


Импортируем необходимые библиотеки:

In [103]:
# Импорт библиотек
import numpy as np # для работы с массивами
import pandas as pd # для работы с DataFrame 
import seaborn as sns # библиотека для визуализации статистических данных
import matplotlib.pyplot as plt # для построения графиков
import plotly.graph_objects as go

%matplotlib inline

Скачиваем данные из интернета

In [104]:
from src.data_uploader import get_data

get_data()

Прочитаем исходные данные:

In [105]:
df = pd.read_csv('data/data.csv')
df.head()

,Well,Por,Perm,AI,Brittle,TOC,VR,Prod
0,1,12.08,2.92,2.80,81.40,1.16,2.31,4165.196191
1,2,12.38,3.53,3.22,46.17,0.89,1.88,3561.146205
2,3,14.02,2.59,4.01,72.80,0.89,2.72,4284.348574
3,4,17.67,6.75,2.63,39.81,1.08,1.88,5098.680869
4,5,17.52,4.57,3.18,10.94,1.51,1.90,3406.132832


## Практика: линейная регрессия по методу наименьших квадратов

Для начала построим простейшую модель линейной регрессии, проанализируем результаты её работы и выберем наиболее значимые факторы для прогнозирования.

В первой части вам предстоит выполнить задания 5.0–5.6. Максимальное количество баллов, которое можно получить, — 9.

### Задание 5.0. (не оценивается)

Проведите небольшой разведывательный анализ, чтобы поближе познакомиться с исходными данными. Постройте несколько визуализаций, которые помогут вам понять, как устроены исходные признаки, а также смогут продемонстрировать взаимосвязь факторов. 

Сделайте промежуточные выводы из проведённого разведывательного анализа.

In [106]:
# Function for build distribution

def get_box_distribution(data: pd.Series, name: str) -> go.Figure:
    """
    Function for build plotly Bar chart for
    df column distribution

    Args:
        data (pd.Series): values for chart
        name (str): name of chart

    Returns:
        go.Figure: plotly object with value
    """
    fig = go.Figure(
        data=[
                go.Box(
                    x=data.values
                    )
            ],
            layout={
                'title': f'Distribution of {name} (box plot)'
            }
    )
    return fig
    
def get_bar_distribution(data: pd.Series, name: str, step: int=1) -> go.Figure:
    """
    Function for build plotly Bar chart for
    df column distribution

    Args:
        data (pd.Series): values for chart
        name (str): name of chart
        step (str): step of discretisation. Default to 1

    Returns:
        go.Figure: plotly object with value
    """
    min_data = int(data.min())
    max_data = int(data.max())+1

    por_freq_dict = {}
    for i in range(min_data, max_data, step):
        por_freq_dict.update({f'from {i} to {i+step}': len(data.loc[(data>=i)&(data<i+1)])})
    fig = go.Figure(
        data=[
            go.Bar(
                x=list(por_freq_dict.keys()),
                y=list(por_freq_dict.values())
                )
        ],
        layout={
            'title': f'Distribution of {name} (bar plot)'
        }
    )
    return fig

#### Распределение пористости скважины
Исходя из-за низкого разброса значений данных (минимальное значение составляло 6 процентов, максимальное - 24 процента) было решено построить гистограмму распределения с шагом в один процент. Это показало, что распределение этого показателя немного скошено вправо (что видно по значениям с 19 до 20). Box plot не выявил каких-либо аномалий в данном ряду.

In [107]:
get_box_distribution(df['Por'], 'well porosity')

In [108]:
get_bar_distribution(df['Por'], 'well porosity')

#### Распределение проницаемости скважины
Следуя той же логике была построена диаграмма распределения проницаемости скважины, которая выявила ряд аномалий: 5 скважин имеют завышеный уровень проницаемости. В целом можно обратить внимание на то, что распределение скошено вправо

In [109]:
get_box_distribution(df['Perm'], 'well permeability')

In [110]:
get_bar_distribution(df['Perm'], 'well permeability')

#### Распределение акустического импеданса
Из данного распределения можно увидеть 2 аномалии с обоих сторон Box plot

In [111]:
get_box_distribution(df['AI'], 'well AI')

In [112]:
get_bar_distribution(df['AI'], 'well AI')

#### Распределение коэффициента хрупкости скважины
На данных графиках мы видим скошенность медианного распределения данной величины, а также низкое количество скважин с показателем от 43 до 46. 

In [113]:
get_box_distribution(df['Brittle'], 'well brittle')

In [114]:
get_bar_distribution(df['Brittle'], 'well brittle', 3)

#### Распределение общего органического углерода
В данном распределении можно увидеть небольшую скошенность медианного распределения

In [115]:
get_box_distribution(df['TOC'], 'well TOC')

In [116]:
get_bar_distribution(df['TOC'], 'well TOC')

#### Распределение коэффициента отражения витринита
В данном распределении можно увидеть 5 аномалий, которые подсвечивает Box plot

In [117]:
get_box_distribution(df['VR'], 'well VR')

In [118]:
get_bar_distribution(df['VR'], 'well VR')

### Задание 5.1. (2 балла)

Постройте корреляционную матрицу факторов, включив в неё целевой признак. 

Для наглядности визуализируйте полученную матрицу с помощью тепловой карты. 

**Примечание.** *Для визуализации вы можете использовать любую из знакомых вам библиотек.*

На основе построенной корреляционной матрицы ответьте на следующий вопрос:

* Какие факторы сильнее всего коррелируют с целевой переменной?

Вычислите ранг и определитель полученной матрицы корреляций и приведите развёрнутые ответы на следующие вопросы:
* Является ли корреляционная матрица плохо обусловенной (близок ли её опредитель к нулю)?
* Что вы можете сказать о наличии коллинераности/мультиколлинеарности в данных? 
* Применима ли в данном случае модель классической линейной регрессии по методу наименьших квадратов и почему? 

**Критерии оценивания:**
- Задание выполнено верно, учтены все условия (**2 балла**): 
    * приведён код для расчёта корреляционной матрицы;
    * приведён код для визуализации корреляционной матрицы в виде тепловой карты;
    * рассчитаны ранг и определитель корреляционной матрицы;
    * предоставлены обоснованные ответы на все поставленные вопросы. 

- Задание выполнено верно, но не учтено одно условие (**1 балл**).
- Задание выполнено неверно, не учтено несколько условий (**0 баллов**).

In [119]:
def label_map(x):
    if abs(x)==1:
        return ''
    elif abs(x)<0.3:
        return 'very low'
    elif abs(x)>=0.3 and abs(x)<0.5:
        return 'low'
    elif abs(x)>=0.5 and abs(x)<0.7:
        return 'medium'
    elif abs(x)>=0.7 and abs(x)<0.9:
        return 'high'
    elif abs(x)>0.9:
        return 'high'

corr_matrix = df.corr() # drop Well column because it's tech
corr_matrix_label_map = corr_matrix.map(label_map)
fig = go.Figure(
    data=[
        go.Heatmap(
            x=corr_matrix.columns,
            y=corr_matrix.columns,
            z=corr_matrix.values,
            text=corr_matrix_label_map,
            texttemplate="%{text}"
        )
    ],
    layout={
        'title': 'Heatmap of DataFrame distribution'
    }
)

fig.show()

In [120]:
rank_corr = np.linalg.matrix_rank(corr_matrix.values)
det_corr = np.linalg.det(corr_matrix.values)

print(f'Ранк корреляционной матрицы: {rank_corr}')
print(f'Определитель корреляционной матрицы: {det_corr}')

Ранк корреляционной матрицы: 8
Определитель корреляционной матрицы: 0.0007299388072652095


- **С целевой переменной более всего коррелируют переменные**:
1) Пористости скважины (0.86 %). Уровень корреляциии - высокий;
2) Проницаемость скважины (0.72 %). Уровень корреляциии - высокий;
3) Общий органический углерод (0.65 %). Уровень корреляциии - средний.
- **Является ли корреляционная матрица плохо обусловенной (близок ли её опредитель к нулю)?**

Определитель матрицы достаточно низок для того, чтобы говорить о том, что проблемы мультиколлинеарности возможны;
- **Что вы можете сказать о наличии коллинераности/мультиколлинеарности в данных?**

Исходя из анализа корреляционной матрицы можно увидеть высокую корреляцию между показателями пористости скважины и проницаемостью скважины (Por и Perm), а также между пористостью скважины и общим органическим углеродом (Por и TOC). Также существует связь между общим органическим углеродом и аустическим импедансом (TOC и AI).
- **Применима ли в данном случае модель классической линейной регрессии по методу наименьших квадратов и почему?**

Учитывая наличие мультиколлинеарности в признаках использование метода линейной регрессии на всех факторах приведет к неверным результатам.

### Задание 5.2. (2 балла)

Создайте матрицу наблюдений `X` и вектор правильных ответов `y`. В качестве факторов для матрицы наблюдений возьмите все имеющиеся в данных признаки. 

Постройте модель линейной регрессии по методу наименьших квадратов. Для этого воспользуйтесь матричной формулой МНК и инструментарием библиотеки numpy. 

Выведите на экран полученные оценки коэффициентов модели, округлённые до целого.

Приведите подробные выводы по полученным результатам, ответив на следующие вопросы:
* Есть ли в ваших данных признаки, которые, согласно модели, можно назвать неинформативными, то есть они не оказывают влияния на целевую переменную или оказывают шумовое влияние?
* Приведите интерпретацию нескольких (двух или трёх) коэффициентов полученной модели линейной регрессии.


**Критерии оценивания:**
- Задание выполнено верно, учтены все условия (**2 балла**): 
    * приведён код для нахождения параметров модели линейной регрессии с помощью матричной формулы МНК;
    * на основе полученных параметров даны верные ответы на поставленные вопросы;
    * приведена корректная интерпретация нескольких коэффициентов полученной модели линейной регрессии.

- Задание выполнено верно, но не учтено одно из условий (**1 балл**).
- Задание выполнено неверно, не учтено несколько условий (**0 баллов**).

In [121]:
X = df.drop(['Prod'], axis=1).values # Исключаем y
X = np.column_stack((np.ones(len(X)), X))
y = df['Prod'].values
w_hat = np.linalg.inv(X.T@X)@X.T@y
print('Оценки линейной регрессии: ', w_hat.round(0))


Оценки линейной регрессии:  [-1232.     0.   230.   116.  -365.    25.   -78.   785.]


- **Есть ли в ваших данных признаки, которые, согласно модели, можно назвать неинформативными, то есть они не оказывают влияния на целевую переменную или оказывают шумовое влияние?**

Назвать такие коэффициенты без использование дополнительных критериев (такие как *t-критерий*) точно не возможно, однако очевидно, что лишним показателем кажется Well
- **Приведите интерпретацию нескольких (двух или трёх) коэффициентов полученной модели линейной регрессии.**

1) 1% пористости скважины увеличивает добычу газа в шахте на 230 млн. кубических футов;
1) 1% коэффициента отражения винирита увеличивает добычу газа в шахте на 783 млн. кубических футов;

### Задание 5.3. (1 балл)

Теперь потренируемся строить предсказание для наблюдений целевой переменной. 

**а)** Постройте прогноз выработки газа для скважины со следующими параметрами:

```python
{
    'Well': 106.0,
    'Por': 15.32,
    'Perm': 3.71,
    'AI': 3.29,
    'Brittle': 55.99,
    'TOC': 1.35,
    'VR': 2.42
 }
```

Рассчитайте абсолютную ошибку построенного вами прогноза для предложенной скважины (в миллионах кубических футов в день), если известно, что на этой скважине производится `4748.315024` миллионов кубических футов газа в день. 

**б)** Постройте прогноз выработки газа для всех скважин из обучающего набора данных. Выберите метрику, по которой вы будете оценивать качество регрессионных моделей и сравнивать их между собой (можно использовать несколько метрик, но не более двух).

Рассчитайте значение выбранной метрики для вашей модели. Приведите интерпретацию полученного результата. 

**Критерии оценивания:**
- Задание выполнено верно, учтены все условия (**1 балл**): 
    * приведён код для построения прогноза выработки газа на указанной скважине и рассчитана абсолютная ошибка прогноза для этой скважины;
    * выбрана корректная для поставленной задачи метрика (или метрики) оценки качества модели и приведён код для расчёта этой метрики на всём обучающем наборе данных;
    * приведена корректная интерпретация полученного результата.

- Задание выполнено верно, но не учтено одно из условий, или задание выполнено неверно (**0 баллов**).

In [122]:
# Task a: one prediction
new = np.array([[1, 106.0, 15.32, 3.71, 3.29, 55.99, 1.35, 2.42]])
y_true = 4748.315024
y_pred = new@w_hat

print('prediction for new well: ', y_pred[-1])
print('Absolute error: ', abs(y_true-y_pred)[-1])

# Task б: prediction for all dataset
y_pred_all = X@w_hat
print('MAPE: ', sum((abs((y-y_pred)/y)))*100/len(y))

prediction for new well:  4723.06405370718
Absolute error:  25.250970292820057
MAPE:  24.491873635620635


1) Абсолютная ошибка прогноза для новой скважины составляет 25.25 млн. кубических футов газа;
2) Выбрана метрика MSPE, т. к. ее наиболее просто интерпретировать. В данном прогнозе она составляет 24.49 %, что говорит о том, что модель ошибается в среднем на 24.49% 

### Задание 5.4. (1 балл)

Настало время анализа построенной модели. Посмотрите на коэффициенты и сравните их знаки со значениями выборочных корреляций между целевым признаком и факторами, которые вы нашли ранее.

Дайте развёрнутые ответы на следующие вопросы:

* Есть ли в вашей модели фактор, при котором коэффициент в модели линейной регрессии противоречит соответствующему коэффициенту корреляции? Например, корреляция говорит, что зависимость между фактором и целевой переменной прямая, а модель говорит обратное. Если таких факторов несколько, укажите их все.
* Как вы считаете, с чем связано полученное противоречие результатов?

**Критерии оценивания:**

- Задание выполнено верно, даны корректные развёрнутые ответы на все поставленные в задании вопросы (**1 балл**). 

- Задание выполнено неверно, даны некорректные ответы на один или несколько поставленных в задании вопросов (**0 баллов**).

In [123]:
w_hat_without_intersept = w_hat[1:]
corr_prod = corr_matrix.loc['Prod'].values[:-1]
labels = corr_matrix.columns[:-1]

for i in zip(w_hat_without_intersept, corr_prod, labels):
    if (i[0]<0 and i[1]>0) or \
        (i[0]>0 and i[1]<0):
        print('Coef with different sign: ', i[2])

Coef with different sign:  TOC


1) Как видно из кода выше, такой признак - это TOC (общий органичекий углерод);
2) Получение противоречивых результатов связано с влиянием признаков друг на друга. Если бы у нас был только 1 признак, то такого бы не произошло.

### Задание 5.5. (2 балла)

* Исключите из данных сильно коррелированные между собой факторы. Под сильной корреляцией в данной задаче будем понимать значения выше `0.7`.

Выбирая, какой из коррелированных факторов оставить, руководствуйтесь коэффициентом корреляции с целевой переменной: оставляйте тот фактор, который больше всего коррелирует с целевой переменной (объёмом добычи газа).

* Также исключите из данных факторы, для которых корреляция с целевой переменной меньше `0.05`.

Постройте модель линейной регрессии на обновлённых после удаления факторов данных по методу наименьших квадратов. Для этого используйте матричную формулу и библиотеку numpy. Выведите значение полученных коэффициентов, округлённых до целого.

Сделайте прогноз для всего обучающего набора данных и рассчитайте выбранную вами ранее метрику (или метрики). 

Дайте подробные ответы на следующие вопросы:

* Приведите интерпретацию нескольких полученных коэффициентов (двух или трёх). Сравните их значения с полученными ранее.
* Приведите интерпретацию полученных значений метрик. Сравните их значения с полученными ранее. 
* Удалось ли вам побороть противоречие коэффициентов линейной регрессии и коэффициентов корреляции?

**Критерии оценивания:**
- Задание выполнено верно, учтены все условия (**2 балла**): 
    * отобраны верные факторы;
    * на основе отобранных факторов построена модель линейной регрессии с помощью матричной формулы МНК и найдены параметры модели, соответствующие каждому из факторов;
    * построен прогноз для всего обучающего набора данных и рассчитано значение метрики (метрика должна быть выбрана корректно);
    * даны ответы на все поставленные в задании вопросы.

- Задание выполнено верно, но не учтено одно из условий (**1 балл**).
- Задание выполнено верно, не учтено несколько из условий (**0 баллов**).

In [124]:
coef_to_delete = ['Perm', 'TOC', 'Well']

X_new = df.drop(['Prod']+coef_to_delete, axis=1).values # Исключаем y
X_new = np.column_stack((np.ones(len(X_new)), X_new))

w_hat_new = np.linalg.inv(X_new.T@X_new)@X_new.T@y
print('Оценки линейной регрессии: ', w_hat_new.round(0))
y_pred_all = X_new@w_hat_new

print('MSPE: ', sum((((y-y_pred_all)/y)**2))*100/len(y))

Оценки линейной регрессии:  [-1835.   293.  -200.    28.   517.]
MSPE:  0.30503364122465904


In [125]:
w_hat_without_intersept_new = w_hat_new[1:]
corr_prod_new = corr_matrix.loc['Prod'][['Por', 'AI', 'Brittle', 'VR']].values
labels_new = ['Por', 'AI', 'Brittle', 'VR']

for i in zip(w_hat_without_intersept_new, corr_prod_new, labels_new):
    n=0
    if (i[0]<0 and i[1]>0) or \
        (i[0]>0 and i[1]<0):
        print('Coef with different sign: ', i[2])
        n+=1
if n==0:
    print('No coef with different sign')

No coef with different sign


- Исходя из условий задачи были удалены критерии Well (меньше 0.05 корреляции с целевым показателем), Perm (большая корреляция с Por при корреляции ниже с целевым показателем) и 'TOC' (большая корреляция с Por при корреляции ниже с целевым показателем)
1) 1% пористости скважины увеличивает добычу газа в шахте на 293 млн. кубических футов (ранее было 230 млн. кубических футов); 
2) 1% коэффициента отражения винирита увеличивает добычу газа в шахте на 517 млн. кубических футов (ранее было 783 млн. кубических футов);
- В настоящий момент прогноз ошибается в среднем на 0.3 % для каждого прогноза;
- Исходя из кода выше можно говорить о том, что разницы в знаках между коэффициентами корреляции и прогнозными коэффициентами не обнаружено.

### Задание 5.6. (1 балл)

Наконец, давайте построим модель линейной регрессии из библиотеки `scikit-learn (sklearn)` и сравним результаты её работы с теми, что нам удалось получить вручную. 

Постройте модель линейной регрессии на обновлённых после удаления факторов данных по методу наименьших квадратов. Для этого воспользуйтесь классом `LinearRegression` из библиотеки `sklearn`. Выведите значения полученных коэффициентов, округлённые до целого.

Сделайте прогноз для всего обучающего набора данных и рассчитайте значение выбранной вами метрики (или метрик).

Сравните результаты библиотечной реализации с тем, что вы получили ранее. 

**Критерии оценивания:**

- Задание выполнено верно, учтены все условия (**1 балл**): 
    * на основе отобранных факторов построена модель линейной регрессии из библиотеки sklearn и найдены параметры модели, соответствующие каждому из факторов;
    * построен прогноз для всего обучающего набора данных и рассчитано значение метрики (метрика должна быть выбрана корректно);
    * приведён вывод о соответствии результатов, полученных вручную и с помощью библиотеки.
- Задание выполнено неверно, не учтено одно или несколько условий (**0 баллов**).

In [126]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error

model = LinearRegression(fit_intercept=False)
model.fit(X_new, y)
print('Коэффициенты линейной регрессии: ', model.coef_.round(0))
y_pred_sklearn = model.predict(X_new)
print('MAPE: ', sum((abs((y-y_pred_sklearn)/y)))*100/len(y))
print('MAPE (sklearn): ', mean_absolute_percentage_error(y, y_pred_sklearn)*100)
linear_regression_mape = mean_absolute_percentage_error(y, y_pred_sklearn)*100


Коэффициенты линейной регрессии:  [-1835.   293.  -200.    28.   517.]
MAPE:  4.044138420436048
MAPE (sklearn):  4.044138420436046


Сравнивая результаты, полученные вручную и при помощи библиотеки можно увидеть, что они равны, что говорит о правильности расчетов

## Практика: полиномиальная регрессия и регуляризация

Мы продолжаем работать над задачей от владельца компании «Газ-Таз-Ваз-Нефть» Василия.

Ранее мы построили модель линейной регрессии, которая прогнозирует выработку газа на скважине. Для этого мы с помощью матрицы корреляций и рассуждений отобрали некоррелированные, значимые для предсказания признаки. **Далее мы будем использовать именно их (см. задание 5.5).**

Мы хотим улучшить результат — уменьшить ошибку прогноза. Для этого мы воспользуемся моделью полиномиальной регрессии третьей степени. Однако теперь мы знаем, что полиномиальным моделям очень легко переобучиться под исходную выборку. Так как данных у нас не так много (всего 200 скважин), то для контроля качества модели мы будем использовать кросс-валидацию. 

Приступим! Выполните задания 8.1–8.5:


In [127]:
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate

### Задание 8.1. (1 балл)

Стандаризируйте признаки с помощью `StandartScaler` из библиотеки `sklearn`. 

Затем сгенерируйте полиномиальные признаки третьего порядка на факторах, которые вы выбрали для обучения моделей. Для этого воспользуйтесь генератором полиномов `PolynomialFeatures` из библиотеки `sklearn`. Параметр `include_bias` установите в значение `False`.

Выведите на экран, сколько факторов у вас получилось после генерации полиномиальных признаков.

**Важно:** стандартизацию необходимо произвести до генерации полиномиальных факторов!

Обучите модель линейной регрессии из библиотеки `sklearn` (`LinearRegression`) на полученных полиномиальных факторах.

Используя кросс-валидацию оцените среднее значение выбранной вами метрики (или метрик) на тренировочных и валидационных фолдах.

Проинтерпретируйте полученные результаты.

**Критерии оценивания:**

- Задание выполнено верно, учтены все условия (**1 балл**): 
    * на основе отобранных факторов сгенерированы полиномиальные признаки третьего порядка;
    * построена модель полиномиальной регрессии (линейной регрессии на полиномиальных признаках);
    * с помощью кросс-валидации оценено среднее значение выбранной студентом метрики (или метрик) на тренировочных и валидационных фолдах (метрика должна быть выбрана корректно).
- Задание выполнено неверно, не учтено одно или несколько из условий (**0 баллов**).

In [128]:
# Create scaller
scaller = StandardScaler()
X_scalled = scaller.fit_transform(X_new)

# Create polynominal model
poly = PolynomialFeatures(degree=3, include_bias=False)
X_poly = poly.fit_transform(X_scalled)

print('Количество факторов для полиномиальной регрессии: ', len(X_poly[0]))

#Create LinearModel
linear_model = LinearRegression().fit(X_poly, y)

# CV
cv_result = cross_validate(
    linear_model,
    X_poly, y,
    scoring='neg_mean_absolute_percentage_error',
    cv=5,
    return_train_score=True
)

poly_train_score=cv_result['train_score'].mean()
poly_test_score=cv_result['test_score'].mean()

print(f'MAPE на тренировочных фолдах: {-round(poly_train_score, 2)*100}')
print(f'MAPE на валидационных фолдах: {-round(poly_test_score, 2)*100}')
print('MAPE без кросс-валидации: ', round(mean_absolute_percentage_error(y, linear_model.predict(X_poly))*100, 2))

Количество факторов для полиномиальной регрессии:  55
MAPE на тренировочных фолдах: 7.000000000000001
MAPE на валидационных фолдах: 8.0
MAPE без кросс-валидации:  5.85


В итоге можно говорить о том, что полиномиальная модель не дала положительного прироста для прогнозирования данных, т. к. MAPE получилась выше, чем в обычном линейном прогнозе. Однако можно говорить о том, что степень полинома подобрана верно, т. к. переобучения не произошло (это видно по схожим ошибкам на тренировочных и валидационных фолдах).

### Задание 8.2. (2 балла)

Теперь попробуем воспользоваться линейной регрессией с регуляризацией. Для начала возьмём $L_1$-регуляризацию.

Обучите модель `Lasso` из библиотеки `sklearn` на полученных полиномиальных факторах, предварительно стандартизировав факторы. 

Коэффициент регуляризации (`alpha`) подберите самостоятельно с помощью любого известного вам метода подбора гиперпаметров.

Используя кросс-валидацию, оцените среднее значение выбранной вами метрики (или метрик) на тренировочных и валидационных фолдах.

Проинтерпретируйте полученные результаты.

**Критерии оценивания:**

- Задание выполнено верно, учтены все условия (**2 балла**): 
    * правильно построена модель полиномиальной регрессии (линейной регрессии на полиномиальных признаках) с регуляризацией (Lasso), учтены условия необходимости масштабирования факторов для построения модели;
    * приведён код для подбора параметра регуляризации (вручную или с помощью библиотечных инструментов);
    * с помощью кросс-валидации оценено среднее значение выбранной студентом метрики (или метрик) на тренировочных и валидационных фолдах (метрика должна быть выбрана корректно).

- Задание выполнено верно, но не учтено одно из условий (**1 балл**).
- Задание выполнено неверно, не учтено несколько условий (**0 баллов**).

In [129]:
alpha_arr = [round(i*0.1, 2) for i in range(1, 1001)] #округляем из-за особенностей Python с float
cv_result_test_arr = []
cv_result_train_arr = []

# Find alpha by Greed search
for alpha in alpha_arr:
    lasso = Lasso(alpha=alpha, max_iter=10000).fit(X_poly, y)
    
    cv_result = cross_validate(lasso, X_poly, y,
                               scoring='neg_mean_absolute_percentage_error',
                               cv=5, return_train_score=True)
    cv_result_test_arr.append(-cv_result['test_score'].mean()*100)
    cv_result_train_arr.append(-cv_result['train_score'].mean()*100)
    
lasso_test_min_score = np.inf
lasso_optimal_alpha = -1

for i, scores in enumerate(cv_result_test_arr):
        if scores<=lasso_test_min_score:
                lasso_test_min_score=scores
                lasso_optimal_alpha = round((i+1)*0.1, 2)
lasso_train_min_score = cv_result_train_arr[i]
print(f'Optimal alpha is: {lasso_optimal_alpha}, valudation score: {round(lasso_test_min_score, 2)} %')

Optimal alpha is: 4.6, valudation score: 2.28 %


Поиск оптимальной *alpha* осуществлялся алгоритмом жадного поиска, т. е. перебором разных возможных состояний *alpha*. В задании указывалось подобрать только этот параметр, поэтому остальные не рассматривались.

Оптимальной *alpha* оказалось значение в 4.6, при данном значении *alpha* удалось достичь ошибки кросс-валидации в 2.28%, что меньше, чем без регуляризации.


### Задание 8.3. (2 балла)

Проделаем то же самое с $L_2$-регуляризацией.

Обучите модель `Ridge` из библиотеки `sklearn` на полученных полиномиальных факторах, предварительно стандартизировав факторы. 

Коэффициент регуляризации (`alpha`) подберите самостоятельно с помощью любого известного вам метода подбора гиперпаметров.

Используя кросс-валидацию оцените среднее значение выбранной вами метрики (или метрик) на тренировочных и валидационных фолдах.

Проинтерпретируйте полученные результаты.

**Критерии оценивания:**

- Задание выполнено верно, учтены все условия (**2 балла**): 
    * правильно построена модель полиномиальной регрессии (линейной регрессии на полиномиальных признаках) с регуляризацией (Ridge), учтены условия необходимости масштабирования факторов для построения модели;
    * приведён код для подбора параметра регуляризации (вручную или с помощью библиотечных инструментов);
    * с помощью кросс-валидации оценено среднее значение выбранной студентом метрики (или метрик) на тренировочных и валидационных фолдах (метрика должна быть выбрана корректно).

- Задание выполнено верно, но не учтено одно из условий (**1 балл**).
- Задание выполнено неверно, не учтено несколько условий (**0 баллов**).

In [130]:
alpha_arr = [round(i*0.1, 2) for i in range(1, 1001)] #округляем из-за особенностей Python с float
cv_result_test_arr = []
cv_result_train_arr = []

# Find alpha by Greed search
for alpha in alpha_arr:
    ridge = Ridge(alpha=alpha, max_iter=10000).fit(X_poly, y)
    
    cv_result = cross_validate(ridge, X_poly, y,
                               scoring='neg_mean_absolute_percentage_error',
                               cv=5, return_train_score=True)
    cv_result_test_arr.append(-cv_result['test_score'].mean()*100)
    cv_result_train_arr.append(-cv_result['train_score'].mean()*100)
    
ridge_test_min_score = np.inf
ridge_optimal_alpha = -1

for i, scores in enumerate(cv_result_test_arr):
        if scores<=ridge_test_min_score:
                ridge_test_min_score=scores
                ridge_optimal_alpha = round((i+1)*0.1, 2)

ridge_train_min_score = cv_result_train_arr[i]
               
print(f'Optimal alpha is: {ridge_optimal_alpha}, valudation score: {round(ridge_test_min_score, 2)} %')

Optimal alpha is: 0.2, valudation score: 2.67 %


Рассматривая L2 регуляризацию можно увидеть, что ошибка прогноза меньше, чем в случае с обычной полиномиальных факторах без регуляризации. Оптимальная alpha - 0.2, при этом ошибка на тестовых фолдах составила 2.67%

### Задание 8.4. (2 балла)

Наконец, настало время комбинировать $L_1$ и $L_2$ -регуляризации.

Обучите модель `ElasticNet` из библиотеки `sklearn` на полученных полиномиальных факторах, предварительно стандартизировав факторы. 

Коэффициенты регуляризации (`alpha` и `l1-ratio`) подберите самостоятельно с помощью любого известного вам метода подбора гиперпаметров.

Используя кросс-валидацию, оцените среднее значение метрики MAPE на тренировочных и валидационных фолдах.

Проинтерпретируйте полученные результаты.

**Критерии оценивания:**

- Задание выполнено верно, учтены все условия (**2 балла**): 
    * правильно построена модель полиномиальной регрессии (линейной регрессии на полиномиальных признаках) с регуляризацией (ElasticNet), учтены условия необходимости масштабирования факторов для построения модели;
    * приведён код для подбора параметра регуляризации (вручную или с помощью библиотечных инструментов);
    * с помощью кросс-валидации оценено среднее значение выбранной студентом метрики (или метрик) на тренировочных и валидационных фолдах (метрика должна быть выбрана корректно).

- Задание выполнено верно, но не учтено одно из условий (**1 балл**).
- Задание выполнено неверно, не учтено несколько условий (**0 баллов**).

In [ ]:
alpha_arr = [round(i*0.1, 2) for i in range(1, 101)] #округляем из-за особенностей Python с float
l1_ratio = [round(i*0.1, 2) for i in range (1, 10)]
params = {'alpha': alpha_arr,
          'l1_ratio': l1_ratio,
          'max_iter': [10000]}

elastic_model = ElasticNet()
grid = GridSearchCV(elastic_model, params, 
                    scoring='neg_mean_absolute_percentage_error').fit(X_poly, y)
best_alpha = grid.best_params_['alpha']
best_l1_ratio = grid.best_params_['l1_ratio']

cv_result = cross_validate(
    ElasticNet(alpha=best_alpha, l1_ratio=best_l1_ratio,
           max_iter=10000), 
    X_poly, y,
    scoring='neg_mean_absolute_percentage_error',
    cv=5, return_train_score=True)

elastic_test_min_score = -cv_result['test_score'].mean()*100
elastic_train_min_score = -cv_result['train_score'].mean()*100

print(f'Optimal alpha is: {best_alpha}, optimal l1_ratio: {best_l1_ratio}, valudation score: {round(elastic_test_min_score, 2)} %')


Optimal alpha is: 0.1, optimal l1_ratio: 0.9, valudation score: 2.76 %


Как видно из предложенных расчетов, модель ElasticNet оказалась хуже других со счетом в 2.76%

### Задание 8.5. (1 балл)

Завершением вашей работы будет сводная таблица результатов, которую вы представите на презентации заказчику Василию.

Составьте таблицу (DataFrame) со следующими столбцами (имена столбцов выберите самостоятельно):
* Наименование модели.
* Гиперпараметры (коэффициенты регуляризации, если таковые имеются), если нет — оставьте ячейку пустой.
* Использовались ли полиномиальные признаки при построении модели (Да/Нет или True/False).
* Значение выбранной метрики на тренировочных фолдах при кросс-валидации модели.
* Значение выбранной метрики на валидационных фолдах при кросс-валидации модели.

Сделайте финальный вывод по проделанной работе. Какая линейная модель из тех, что мы рассмотрели, показала наилучший результат с точки зрения качества на валидационных фолдах, а также с точки зрения переобученности?

**Критерии оценивания:**

- Задание выполнено верно, учтены все условия (**1 балл**): 
    * верно составлена сводная таблица итоговых результатов построенных ранее моделей с указанными в задании столбцами;
    * приведены итоговые выводы по проделанной работе, указано, какая из рассмотренных моделей показала наилучший результат.

- Задание выполнено неверно, не учтено одно или несколько условий (**0 баллов**).

In [136]:
model_names = ['Linear regression', 'Polynomial regression',
               'Lasso', 'Ridge', 'Elastic Net']
hyperparameter_arr = ['-', '-', f'alpha: {lasso_optimal_alpha}', 
                      f'alpha: {ridge_optimal_alpha}',
                      f'alpha: {best_alpha}, l1_ratio: {best_l1_ratio}']
poly_factors = [False, True, True, True, True]
metrics_test = [round(linear_regression_mape, 2), -round(poly_test_score, 2)*100, 
                round(lasso_test_min_score, 2), round(ridge_test_min_score, 2),
                round(elastic_test_min_score, 2)]
metrics_train = [round(linear_regression_mape, 2), -round(poly_train_score, 2)*100,
                 round(lasso_train_min_score, 2), round(ridge_train_min_score, 2),
                 round(elastic_train_min_score, 2)]

df_final = pd.DataFrame(zip(model_names, hyperparameter_arr, 
                            poly_factors, metrics_test, metrics_train),
                        columns=['Наименование моделей', 'Гиперпараметры',
                                 'Использование полиномов', 'Тестовые фолды', 
                                 'Тренировочные фолды'])
df_final

,Наименование моделей,Гиперпараметры,Использование полиномов,Тестовые фолды,Тренировочные фолды
0,Linear regression,-,False,4.04,4.04
1,Polynomial regression,-,True,8.00,7.00
2,Lasso,alpha: 4.6,True,2.28,5.06
3,Ridge,alpha: 0.2,True,2.67,6.69
4,Elastic Net,"alpha: 0.1, l1_ratio: 0.9",True,2.76,1.88


По итогам исследования можно увидеть, что лучше всех себя показала модель Lasso, т. к. она имеет низкое переобучение и лучший результат.